In [1]:
from bs4 import BeautifulSoup
import urllib2
import requests
import pandas as pd
import re
import time
import numpy as np
import json

In [2]:
all_cand = "http://myneta.info/ls2014/index.php?action=summary&subAction=candidates_analyzed&sort=candidate#summary"
winners_crim = "http://myneta.info/ls2014/index.php?action=summary&subAction=winner_crime&sort=candidate#summary"
source=requests.get(winners_crim)

In [3]:
tree= BeautifulSoup(source.text,"html.parser")

In [4]:
len(tree)

22

In [5]:
tree.title

<title>Winners with declared criminal cases:Loksabha 2014 Election</title>

In [6]:
all_links = tree.find_all('a')

In [7]:
table_pol = tree.findAll('table')[2]
rows = table_pol.findAll("tr")[2:]
print rows[10:13]

[<tr><td>11</td><td><a href="candidate.php?candidate_id=6995">Anbumani Ramadoss</a><b></b></td><td>DHARMAPURI</td>
<td>PMK</td><td align="center" style=""><span style="color:red;font-weight:bold;font-size:150%">8</span></td><td>Graduate Professional</td>
<td align="right">Rs 31,43,78,451<br><span style="color:black;font-size:70%;text-decoration:none;white-space:nowrap"> ~ 31 Crore+</span></br></td><td align="right">Rs 4,19,41,065<br><span style="color:black;font-size:70%;text-decoration:none;white-space:nowrap"> ~ 4 Crore+</span> </br></td></tr>, <tr><td>12</td><td><a href="candidate.php?candidate_id=3373">Anil Shirole</a><b></b></td><td>PUNE</td>
<td>BJP</td><td align="center" style=""><span style="color:red;font-weight:bold;font-size:150%">2</span></td><td>Graduate</td>
<td align="right">Rs 22,76,15,279<br><span style="color:black;font-size:70%;text-decoration:none;white-space:nowrap"> ~ 22 Crore+</span></br></td><td align="right">Rs 47,81,007<br><span style="color:black;font-size:70

In [8]:
# We build one dictionary per candidate #
# We give each candidates to idenfiers: its order in the list and its url address

def id2(r):
    url_string = str(r.find("a").get("href"))
    id2 = int(re.search(r'\d+', url_string).group())
    return id2

def c_link(r):
    return r.find("a").get("href")
def name(r):
    return r.find("a").get_text()
def cols(r):
    return r.findAll("td")
def assets(r):
    col = cols(r)
    ass1 = col[6].get_text().split("~")[0].encode('ascii', 'ignore').replace("Rs","").replace(",","")
    if ass1 == "Nil":
        ass2 = 0
    else:
        ass2=int(ass1)
    return ass2

def liab(r):
    col = cols(r)
    liab1 = col[7].get_text().split("~")[0].encode('ascii', 'ignore').replace("Rs","").replace(",","")
    if liab1 == "Nil":
        liab2 = 0
    else:
        liab2 = int(liab1)
    return liab2

info_candidate = lambda r: [int(cols(r)[0].get_text()),id2(r), r.find("a").get("href"),r.find("a").get_text(),
                            cols(r)[2].get_text(),cols(r)[3].get_text(),cols(r)[5].get_text(),
                            int(cols(r)[4].get_text()),assets(r), liab(r)]

title = ['id','id2','url','name','district','party','education','nr_crime','assets','liabilities']
dict_candidates = [dict(zip(title,info_candidate(r))) for r in rows]

In [9]:
len(dict_candidates)
dict_candidates[0:5]

[{'assets': 140831576,
  'district': u'MAHBUBNAGAR',
  'education': u'Graduate',
  'id': 1,
  'id2': 4536,
  'liabilities': 1386969,
  'name': u'A P Jithender Reddy',
  'nr_crime': 4,
  'party': u'TRS',
  'url': u'candidate.php?candidate_id=4536'},
 {'assets': 17791149,
  'district': u'ATTINGAL',
  'education': u'Doctorate',
  'id': 2,
  'id2': 340,
  'liabilities': 1421392,
  'name': u'A Sampath',
  'nr_crime': 1,
  'party': u'CPI(M)',
  'url': u'candidate.php?candidate_id=340'},
 {'assets': 15765250,
  'district': u'CUDDALORE',
  'education': u'Graduate Professional',
  'id': 3,
  'id2': 3955,
  'liabilities': 0,
  'name': u'A.Arunmozhithevan',
  'nr_crime': 9,
  'party': u'AIADMK',
  'url': u'candidate.php?candidate_id=3955'},
 {'assets': 253880475,
  'district': u'SHIRUR',
  'education': u'Others',
  'id': 4,
  'id2': 3391,
  'liabilities': 61810866,
  'name': u'Adhalrao Shivaji Dattatrey',
  'nr_crime': 10,
  'party': u'SHS',
  'url': u'candidate.php?candidate_id=3391'},
 {'assets

In [10]:
# Now we create a really big dictionary which stores url and page for each candidate
# Work in progress...
# First transform the thing into a dataframe

df_pol = pd.DataFrame(dict_candidates)
order_cols = ['id2','name','district','party','education','assets','liabilities','nr_crime','url']
df_pol = df_pol[order_cols].sort(['assets'],ascending=0)

df_small=df_pol[["id2","url"]][0:3] # I use df_small to test the code before running it on 8400 candidates!
df_small
df_pol.head()

#Note already that the same guy can be candidate at 2 districts #
# Might be a problem

,id2,name,district,party,education,assets,liabilities,nr_crime,url
59,9455,Gokaraju Ganga Raju,NARSAPURAM,BJP,Graduate,2883567122,168501286,1,candidate.php?candidate_id=9455
30,8202,Butta Renuka,KURNOOL,Yuvajana Sramika Rythu Congress Party,10th Pass,2426240513,312279927,1,candidate.php?candidate_id=8202
160,88,Shatrughan Sinha,PATNA SAHIB,BJP,Graduate,1317405961,168760929,1,candidate.php?candidate_id=88
164,8239,Srinivas Kesineni,VIJAYAWADA,TDP,10th Pass,1284122669,715462989,3,candidate.php?candidate_id=8239
32,6088,Captain Amarinder Singh,AMRITSAR,INC,Graduate,863564071,4853456,3,candidate.php?candidate_id=6088


In [11]:
urlcache={}

In [12]:
def get_page(url):
    # Check if URL has already been visited.
    if (url not in urlcache) or (urlcache[url]==1) or (urlcache[url]==2):
        time.sleep(1)
        print len(urlcache)
        # try/except blocks are used whenever the code could generate an exception (e.g. division by zero).
        # In this case we don't know if the page really exists, or even if it does, if we'll be able to reach it.
        try:
            r = requests.get("http://myneta.info/ls2014/%s" % url)

            if r.status_code == 200:
                urlcache[url] = r.text
            else:
                urlcache[url] = 1
        except:
            urlcache[url] = 2
    return urlcache[url]

In [13]:
df_pol["url"].apply(get_page) # This is a very long call
                              # I am saving it in order to run it only once

59     \t\t\t<html>\n\t\t\t\t<head>\n\t\t\t\t   <titl...
30     \t\t\t<html>\n\t\t\t\t<head>\n\t\t\t\t   <titl...
160    \t\t\t<html>\n\t\t\t\t<head>\n\t\t\t\t   <titl...
164    \t\t\t<html>\n\t\t\t\t<head>\n\t\t\t\t   <titl...
32     \t\t\t<html>\n\t\t\t\t<head>\n\t\t\t\t   <titl...
39     \t\t\t<html>\n\t\t\t\t<head>\n\t\t\t\t   <titl...
127    \t\t\t<html>\n\t\t\t\t<head>\n\t\t\t\t   <titl...
16     \t\t\t<html>\n\t\t\t\t<head>\n\t\t\t\t   <titl...
24     \t\t\t<html>\n\t\t\t\t<head>\n\t\t\t\t   <titl...
162    \t\t\t<html>\n\t\t\t\t<head>\n\t\t\t\t   <titl...
146    \t\t\t<html>\n\t\t\t\t<head>\n\t\t\t\t   <titl...
65     \t\t\t<html>\n\t\t\t\t<head>\n\t\t\t\t   <titl...
121    \t\t\t<html>\n\t\t\t\t<head>\n\t\t\t\t   <titl...
41     \t\t\t<html>\n\t\t\t\t<head>\n\t\t\t\t   <titl...
21     \t\t\t<html>\n\t\t\t\t<head>\n\t\t\t\t   <titl...
166    \t\t\t<html>\n\t\t\t\t<head>\n\t\t\t\t   <titl...
91     \t\t\t<html>\n\t\t\t\t<head>\n\t\t\t\t   <titl...
105    \t\t\t<html>\n\t\t\t\t<h

In [14]:
print np.sum([(urlcache[k]==1) or (urlcache[k]==2) for k in urlcache])# no one or 0's
print len(df_pol.url.unique())==len(urlcache)#we got all of the urls

0
True


In [15]:
with open("tempdata/polinfo.json","w") as fd:
    json.dump(urlcache, fd)
del urlcache

In [16]:
with open("tempdata/polinfo.json") as json_file:
    pol_pages = json.load(json_file) # Next iterations will start from here!

In [31]:
# Now each candidate has an url - so later we  will parse each of their pages and scrape them#
#But first let's look how the page looks like for a few individual candidates
pages_list = [pol_pages[x] for x in df_pol['url'][0:10]] # List of 10 that we can start looking at

page_candidate = pages_list[8]
c_soup = BeautifulSoup(page_candidate,"html.parser")


In [32]:
#### Personal info #########

personal = c_soup.findAll(attrs={"class": "grid_3 alpha"})[0]
full_name = personal.find("h2").get_text()
district1 = personal.find("h5").get_text()
grid2 = personal.findAll(attrs={"class":"grid_2 alpha"})
party_full = grid2[0].get_text().split(":")[1]
age = int(grid2[2].get_text().split(":")[1])
address = grid2[3].get_text().split(":")[1].split("\n")[1] # Careful this one changes
self_profession = personal.find("p").get_text().split('\n')[0].split(":")[1]
spouse_profession = personal.find("p").get_text().split('\n')[1].split(":")[1]
print full_name, "\n works as a:", self_profession, ". \n Profession of spouse:", spouse_profession, ". \n Address: ",
print address, ", in district: ", district1, "for party:", party_full, "The dude is", age, "years old"

########## Assets  & Criminality ###########
# Ok let's get serious and find his assets and criminal activity
all_tables = c_soup.findAll("table")
print "\n \n \n" , len(all_tables)
table_inc = all_tables[0]
table_crim = all_tables[1]
table_assets1 = all_tables[2]
table_assets2 = all_tables[3]
table_liab = all_tables[4]
#print table_inc
# That does not really work: some have a table for criminal activities, others not
# So we need more work to know exactly how to have the right table correspond to the right thing


BARNE SHRIRANG CHANDU (Winner) 
 works as a: Farming & Construction Business . 
 Profession of spouse: Farming & Transport Business . 
 Address:              Shri Nivas, Near Padmaji Paper Mill, Thergaon, Pune 33 , in district:  
        MAVAL  (MAHARASHTRA)
         for party: SHS
  The dude is 50 years old

 
 
8
